<a href="https://colab.research.google.com/github/MatteoRigoni/MachineLearningPlayground/blob/master/MachineLearning_Modelli_Algoritmi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Gradient Descent (teoria)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from time import time
from sklearn.utils import shuffle

RANDOM_SEED = 0

In [ ]:
#creazione di un dataset di esempio e divisione training/test...
from sklearn.datasets import make_regression
x, y = make_regression(n_samples=100, n_features=50, bias=5., noise=20., random_state=RANDOM_SEED)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.3, random_state=RANDOM_SEED)

In [ ]:
from sklearn.linear_model import SGDRegressor #stocastics, se vogliamo full mettiamo batch_size uguale ampiezza dataset

# se si ottiene "ConvergenceWarning" indica che abbiamo finito le epoche prima della conservenza.
model = SGDRegressor(max_iter=1000)
model.fit(x_train, y_train) #addestramento

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

#funzione di aiuto valutazione modello
def evaluate(model, data):
  x,y = data
  y_pred = model.predict(x)
  print(f"MSE={mean_squared_error(y, y_pred)}")
  print(f"R2={r2_score(y, y_pred)}")

In [ ]:
#valutiamo il modello...
evaluate(model, (x_train, y_train))
evaluate(model, (x_test, y_test))

#R molto alto, potrebbe esserci overfitting, infatti MSE molto alto nel test rispetto al train
#risciviamo con penalizzazione, usando L1+L2 pesate=elasticnet
#  usiamo learning rate adattivo, che varia durante le varie epoche, quindi gli step finale saranno più piccoli e precisi
#  aumentiamo max_iter per non avere più warning
model = SGDRegressor(max_iter=5000, penalty="elasticnet", alpha=0.01, l1_ratio=0.9, learning_rate="adaptive")

model.fit(x_train, y_train) #addestramento

#rivalutiamo il modello...
evaluate(model, (x_train, y_train))
evaluate(model, (x_test, y_test))
#c'è ancora overfitting in parte, ma se non è cambiato nulla, vuol dire che abbiamo già raggiunto migliori performance...

### Modello di classificazione con Stocasting Descent

Learning rate puà avere come valori: constant con valore di eta0, optimal, invscaling di default

In [ ]:
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score, log_loss

RANDOM_SEED = 0

#dataset di esempio
x, y = make_classification(n_samples=100, n_features=30, n_informative=30, n_redundant=0, n_repeated=0, n_classes=2, random_state=RANDOM_SEED)

In [ ]:
#funzione di valutazione
from sklearn.metrics import mean_squared_error, r2_score

def evaluate(model, data):
  x,y = data
  y_pred = model.predict(x)
  print(f"Accuracy={accuracy_score(y, y_pred)}")
  print(f"Log Loss={log_loss(y, y_pred)}")

In [ ]:
from sklearn.linear_model import SGDClassifier # simile a quanto fatto con regressione lineare..

#divisione dataset train, test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.3, random_state=RANDOM_SEED)

model = SGDClassifier()
model.fit(x_train, y_train)
evaluate(model, (x_train, y_train))
evaluate(model, (x_test, y_test))

#sembra esserci overfitting, regolizziamo...
model = SGDClassifier(penalty="elasticnet", alpha=0.01, l1_ratio=0.8, learning_rate="constant", eta0=10)
model.fit(x_train, y_train)
evaluate(model, (x_train, y_train))
evaluate(model, (x_test, y_test))
#peggiora, quindi lasciamo così

model = SGDClassifier(learning_rate="constant", eta0=10) # se troppo alto il modello rischia di rimbalzare, senza trovare il minimo
model.fit(x_train, y_train)
evaluate(model, (x_train, y_train))
evaluate(model, (x_test, y_test))

###Mini batch Gradient Descent

Di default sklearn analizza un esempio per volta, col il mini, possiamo lavorare a porzioni di dati

In [ ]:
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score, log_loss

RANDOM_SEED = 0

#dataset di esempio
x, y = make_classification(n_samples=100, n_features=30, n_informative=30, n_redundant=0, n_repeated=0, n_classes=2, random_state=RANDOM_SEED)

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.utils import shuffle

#epochs = 5
#epochs = 350 #full
epochs = 70 #mini batch
#n_batches = x_train.shape[0] # facendo così abbiamo lo stocatics
#n_batches = 1 #full
n_batches = 12

batch_size = x_train.shape[0]/n_batches

classes = np.unique(y_train) #abbiamo due classi essendo binaria

sgd = SGDClassifier(loss="log") #funzione di costo log loss
sgd_loss = []

tick = time()

for epoch in range(epochs): #alla fine di ogni epoca dobbiamo mischiare i dati
  x_shuffled, y_shuffled = shuffle(x_train, y_train)
  for batch in range(n_batches):
    batch_start = int(batch*batch_size)
    batch_end = int((batch+1)*batch_size)

    x_batch = x_shuffled[batch_start:batch_end, :]
    y_batch = y_shuffled[batch_start:batch_end]

    #step di epoca di addestramento sul batch
    sgd.partial_fit(x_batch, y_batch, classes=classes) #vanno passate classi perchè non è detto che ci siano tutte nel batch
    #calcoliamo log loss...
    loss = log_loss(y_train, sgd.predict_proba(x_train), labels=classes)
    sgd_loss.append(loss)

  print(f"Loss all'epoca {epoch+1} = {loss}")

print(f"Addestramento completato in {time()-tick:.2f} secondi")

In [ ]:
#mostriamo risultato graficamente...
def train_history(losses, title):
  plt.figure(figsize=(14,10))
  plt.title(title)
  plt.xlabel("Iterazione") #che non è l'epoca, ma ogni step, con x_train=7 e epoch=5, saranno 350 iterazioni
  plt.ylabel("Log-Loss")
  plt.plot(losses)

train_history(sgd_loss, "Stochasting Gradient Descend")

#c'è rumore nella fase di addestramento, si riprova con 350 epoche e batch=1
#dopo 100 epoche modello ha raggiunto convergenza e rimane lineare
#poi proviamo con batch size di 12 (mini batch) e numero epoche 70, così via di mezzo, c'è un pò di rumore, ma modello migliora linearmente

###Early stopping

Tecnica che permette di stoppare il modello se ha smesso di migliorare, ovvero di diminuire dopo tot epoche

Esiste anche come parametro in SGDRegression

In [ ]:
#refactoring del codice precedente:

from sklearn.linear_model import SGDClassifier
from sklearn.utils import shuffle

epochs = 70
n_batches = 12

#se dopo 5 iterazioni, il costo non è migliorato di almento 0.0001 stoppiamo addestramento
tol = 0.0001
n_iter_no_change = 5
n_iter_count = 0
best_loss = 100 #valore migliore fino a quel momento

batch_size = x_train.shape[0]/n_batches

classes = np.unique(y_train) #abbiamo due classi essendo binaria

sgd = SGDClassifier(loss="log") #funzione di costo log loss
sgd_loss = []

tick = time()

for epoch in range(epochs): #alla fine di ogni epoca dobbiamo mischiare i dati
  x_shuffled, y_shuffled = shuffle(x_train, y_train)
  for batch in range(n_batches):
    batch_start = int(batch*batch_size)
    batch_end = int((batch+1)*batch_size)

    x_batch = x_shuffled[batch_start:batch_end, :]
    y_batch = y_shuffled[batch_start:batch_end]

    #step di epoca di addestramento sul batch
    sgd.partial_fit(x_batch, y_batch, classes=classes) #vanno passate classi perchè non è detto che ci siano tutte nel batch
    #calcoliamo log loss...
    loss = log_loss(y_train, sgd.predict_proba(x_train), labels=classes)
    sgd_loss.append(loss)

  #valutazione alla fine di ogni epoca...
  if loss >= best_loss-tol:
    if n_iter_count >= n_iter_no_change:
      print("Early Stopping!!!")
      break
    else:
      n_iter_count+=1
  else: # nuovo valore migliore..
    best_loss= loss
    n_iter_count=0

  print(f"Loss all'epoca {epoch+1} = {loss} ({best_loss-loss})")

print(f"Addestramento completato in {time()-tick:.2f} secondi")

#Gradient Descent (esercitazione)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model  import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics  import log_loss
from sklearn.utils import shuffle
from sklearn.metrics import classification_report, accuracy_score, log_loss

In [ ]:
RANDOM_SEED = 0
BASE_URL = "https://raw.githubusercontent.com/ProfAI/machine-learning-modelli-e-algoritmi/main/datasets/"

In [ ]:
#diagnosis è la variabile target che indica se il tumore è maligno/benigno
#sappiamo già che il dataset non ha difetti e saltiamo la fase di analisi dei dati

df = pd.read_csv(BASE_URL + "breast_cancer.csv")
df.count()
df.head()

In [ ]:
map_dict = {"M":1, "B":0}
df["diagnosis"] = df["diagnosis"].map(lambda x: map_dict[x])
df.head()

In [ ]:
#preparazione array di valutazione
x= df.drop(["diagnosis", "ID number"], axis=1).values
y = df["diagnosis"].values

#hold out
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.3, random_state=RANDOM_SEED)

#riportiamo variabili su stessa scala...
ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.transform(x_test)

In [ ]:
#fase di addestramento
def fit(model, x, y, batch_size=1, epochs=100, verbose=True): # default batch-size=1, quindi stochatics  descend
  n_batches = int(x.shape[0]/batch_size)+1
  classes = np.unique(y) # potrebbero mancare classi in quel batch

  batch_loss = []

  for epoch in range(epochs):
    x_shuffled, y_shuffled = shuffle(x, y)
    for batch in range(n_batches):
      batch_start = int(batch*batch_size)
      batch_end = int((batch+1)*batch_size)
      x_batch = x_shuffled[batch_start:batch_end, :]
      y_batch = y_shuffled[batch_start:batch_end]
      model.partial_fit(x_batch, y_batch, classes = classes)
      loss = log_loss(y_test, model.predict_proba(x_test), labels=classes)
      batch_loss.append(loss)

    if verbose:
      print(f"Loss all'epoca {epoch+1} ) {loss}")

  return model

def evaluate(model, x, y, label=None):
  y_pred = model.predict(x)
  y_proba = model.predict_proba(x)

  if label is not None:
    print(label)

  accuracy = accuracy_score(y, y_pred)
  loss = log_loss(y, y_proba)

  print(f"Accuracy={accuracy} Log Loss = {loss:.3f}")
  return accuracy, loss

In [ ]:
#addrestiamo con differenti batch_size...
batch_sizes = [8, 16, 32, 64, 128]

best_model = None
best_loss = 100

for batch_size in batch_sizes:
  print(f"Batch size = {batch_size}")
  sgd = SGDClassifier(loss="log_loss")
  fit(sgd, x_train, y_train, batch_size=batch_size, epochs=200, verbose=False)
  _, loss = evaluate(sgd, x_test, y_test, label="TEST SET")

  if best_model is None or loss < best_loss:
    best_model = sgd
    best_loss = loss

  #consideriamo migliore accuracy e loss più basso

print(classification_report(y_train, best_model.predict(x_train)))
print(classification_report(y_test, best_model.predict(x_test))) #vediamo se mantiene performance sul set di test:Sì


In [ ]:
#valutazione di un nuovo dataset che è stato fornito
df_update = pd.read_csv(BASE_URL+"breast_cancer_update.csv")
df_update["diagnosis"] = df_update["diagnosis"].map(lambda x:map_dict[x])

x_update = df.drop(["diagnosis", "ID number"], axis=1).values
y_update = df["diagnosis"].values

#standardizzazione con lo stesso modello usato durante l'addestramento
#(in un caso reale avremmo dovuto esportare il precedente modello e reimportarlo in un nuovo notebbok)
x_update = ss.transform(x_update)
best_model.partial_fit(x_update, y_update)

#come metriche non è cambiato nulla
print(classification_report(y_test, best_model.predict(x_test)))


#Naive Bayes (teoria)

###Gaussian Naive Bayes

Assume che la distrubuizione della probabilità x dato y, sia gaussiana (con maggioranza attorno la media)


In [ ]:
from sklearn.datasets import load_iris, make_gaussian_quantiles, make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, classification_report, confusion_matrix
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
RANDOM_SEED = 0

In [ ]:
x, y = make_gaussian_quantiles(n_features=1, n_classes=2, random_state=RANDOM_SEED) #dataset di prova
plt.scatter(np.arange(x.shape[0]), x, c=y) # si vede distribuzione attorno alla media che è zero

In [ ]:
#proviamo prima semplice algoritmo regressione logistica per la classdificazione
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.3, random_state=RANDOM_SEED) # non serve standardizzare, lo è per definizione di gaussiana

lr = LogisticRegression()
lr.fit(x_train, y_train)
lr.score(x_test, y_test) #accuracy di 0.37, estremamente basso

a = np.arange(x.shape[0])
plt.scatter(a, x, c=y)
plt.plot(a, a*lr.coef_[0] + lr.intercept_) # mostriamo anche retta di intercetta

#un sempice modello lineare non sarebbe mai potuto andare meglio di un 50%, in quanto una retta non puà classificare bene questo dataset, dato che segue una gaussiana

In [ ]:
#proviamo quindi altro..

from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
gnb.fit(x_train, y_train)
gnb.score(x_test, y_test) #accuracy del 0.93 sul test
y_proba = gnb.predict_proba(x_test)
log_loss(y_test, y_proba) #0.19, piccola quindi il modello si comporta bene

### Bernoulli

Da usare per feature binarie, ad esempio sui testi, bag of words, creiamo vocabolario (un array) con tutte le parole. Poi per ogni parola inseriamo un 1 nell'array creato alla posizione corrispondente.

Esempio: riconiscimento sms di spam


In [ ]:
from sklearn.datasets import load_iris, make_gaussian_quantiles, make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, classification_report, confusion_matrix
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
RANDOM_STATE = 0
BASE_URL = "https://raw.githubusercontent.com/ProfAI/machine-learning-modelli-e-algoritmi/main/datasets/spam.csv"

In [ ]:
df = pd.read_csv(BASE_URL)
df.head() #1° colonna è un id, non ci importa

In [ ]:
#costruzione del vocabolario per il bag of words
def build_vocab(corpus):
  vocab = []
  for doc in corpus:
    for word in doc.split():
      if word not in vocab:
        vocab.append(word.lower())

  return vocab

# i cicli in python sono molto efficienti, proviamo strada alternativa...
def build_vocab_opt(corpus):
  vocab = set({})
  for doc in corpus:
    vocab = vocab.union(set(doc.lower().split()))
  return list(vocab)

corpus = ["Il cane gioca con la palla", "Il sole brilla nel cielo"]
build_vocab(corpus)
build_vocab_opt(corpus) #ordine diverso ma non conta

In [ ]:
def binary_bow(corpus, vocab=None):
  if vocab is None:
    vocab = build_vocab(corpus)

  vocab_size = len(vocab)
  docs_bow = []

  for doc in corpus:
    doc_bow = [0]*vocab_size # inizializziamo un array con tutti zero
    for i in range(vocab_size): # per ogni token del vocabolario...
      doc_bow[i] = int(vocab[i] in doc)
    docs_bow.append(doc_bow)

  return docs_bow

#esempio
bow = binary_bow(corpus)
bow

In [ ]:
sms_list = df["MESSAGE"].to_list()
sms_list[:5]

sms_bow = binary_bow(sms_list)
len(sms_bow[0])

x = sms_bow
y= df["SPAM"]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.3, random_state=RANDOM_STATE)


In [ ]:
from sklearn.naive_bayes import BernoulliNB

benb = BernoulliNB()
benb.fit(x_train, y_train)
report = classification_report(benb.predict(x_test), y_test)
print(report) #f1-score che è cumulativo degli altri, molto alto, bene

###Multinomial Naive Bayes

Rispetto Bernoulli qui usiamo bow originale, dove non si indica solo presenza parola, ma anche il conteggio, quindi usiamo versione multinomiale, con iperparametro alpha

In [ ]:
RANDOM_STATE = 0
BASE_URL = "https://raw.githubusercontent.com/ProfAI/machine-learning-modelli-e-algoritmi/main/datasets/spam.csv"

def build_vocab_opt(corpus):
  vocab = set({})
  for doc in corpus:
    vocab = vocab.union(set(doc.lower().split()))
  return list(vocab)

def bow(corpus, vocab=None):
  if vocab is None:
    vocab = build_vocab_opt(corpus)

  vocab_size = len(vocab)
  docs_bow = []

  for doc in corpus:
    doc_bow = [0]*vocab_size # inizializziamo un array con tutti zero
    for i in range(vocab_size): # per ogni token del vocabolario...
      doc_bow[i] = doc.split().count(vocab[i])  #dobbiamo inserire quante volte la parola è presente, non solo se c'è o meno come nel precedente "binario"
    docs_bow.append(doc_bow)

  return docs_bow

corpus = ["Il cane gioca con la palla", "Il sole brilla nel cielo con la palla"]
bow(corpus)

In [ ]:
sms_bow = bow(sms_list)

x = sms_bow
y = df["SPAM"].to_list()
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.3, random_state=RANDOM_STATE)

from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB

mnb = MultinomialNB()
mnb.fit(x_train, y_train)
print(classification_report(y_test, mnb.predict(x_test))) #migliore su precision e recall rispetto a Bernoulli, ma simili

#ma se usassimo Bernoulli su dataset non binario?
bnb = BernoulliNB()
bnb.fit(x_train, y_train)
print(classification_report(y_test, bnb.predict(x_test)))

#se i dati non sono binari, Bernoulli li "binarizza", quindi otteniamo stesso risultato
#normliazza tutti i valori tra 0 e 1, portandoli a 1

### Complement Naive Bayes

Variante del multinomial, utile per dataset sbilanciati, quindi con una delle calssi del target molto più presente dell'altra

In [ ]:
RANDOM_STATE = 0
BASE_URL = "https://raw.githubusercontent.com/ProfAI/machine-learning-modelli-e-algoritmi/main/datasets/spam_unbalanced.csv"

df = pd.read_csv(BASE_URL)
df.head()
df["SPAM"].value_counts() #molto sbilanciato, maggioranza di messaggi non sono di spam

In [ ]:
#usiamo funzione bow di sklearn...
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
x = df["MESSAGE"]
y = df["SPAM"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.3, random_state=RANDOM_STATE)

bow = CountVectorizer(stop_words="english", max_features=1000) #usiamo le 1000 più comuni nel dizionario

x_train = bow.fit_transform(x_train)
x_test = bow.transform(x_test)
x_train.shape # 70%,30%

In [ ]:
from sklearn.naive_bayes import ComplementNB

comNB = ComplementNB()
comNB.fit(x_train, y_train)
print(classification_report(y_test, comNB.predict(x_test)))
#precision bassa, recall alta, quindi il modello sta classificando troppe cose come appartenenti alla classe positiva
#vediamo confusion matrix per vedere se è così...
confusion_matrix(y_test, comNB.predict(x_test)) # molti falsi positivi, 18, mentre i falsi negativi sono solo 3

#vediamo come sarebbe andata con il multinomial, che non è fatto per modelli sbilanciati
mnb = MultinomialNB()
mnb.fit(x_train, y_train)
print(classification_report(y_test, comNB.predict(x_test)))
#sembra comportarsi meglio, anche se il dataset è sbilanciato, non c'è sempre una regola empirica

###Categorical Naive Bayes

Variante per variabili di tipo categorico

In [ ]:
RANDOM_STATE = 0
BASE_URL = "https://raw.githubusercontent.com/ProfAI/machine-learning-fondamenti/main/datasets/"

df = pd.read_csv(BASE_URL + "housing.csv", usecols=["ZN", "CHAS", "RAD", "PRICE"])
df.head()

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

x= df.drop("PRICE", axis=1).values
y = df["PRICE"].values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.3, random_state=RANDOM_STATE)

#NOTA
#OrdinalEncoder potrebbe dare errore se nel test compaiono categorie non presenti in quello di train, qundi usiamo apposito parametro
ordenc = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=np.nan)
x_train = ordenc.fit_transform(x_train)
x_test = ordenc.transform(x_test)

#creazione del modello
from sklearn.naive_bayes import GaussianNB, CategoricalNB
gnb = GaussianNB()
gnb.fit(x_train, y_train)
gnb.score(x_test, y_test)
#dà errore, in quanto naive bayes non è indicato per regressioni ma classificazioni
#riformuliamo il dataset per avere delle categorie fisse...

#usiamo quantili per definire features...
df["PRICE"].quantile(0)
df["PRICE"].quantile(1)
#...

def price_to_category(price):
  CATEGORIES = ["VERY CHEAP", "CHEAP", "AVERAGE", "EXPENSIVE", "VERY EXPENSIVE"]

  for i in range(1,5):
    if price < df["PRICE"].quantile(0.2*i): #diviso in 5, quindi andiamo a step di 0.2
      return CATEGORIES[i]

  return CATEGORIES[-1]

#così abbiamo trasformato problema di regressione in classificaizone

df["PRICE"] = df["PRICE"].apply(price_to_category)
df.head()

x= df.drop("PRICE", axis=1).values
y = df["PRICE"].values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.3, random_state=RANDOM_STATE)

ordenc = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=np.nan)
x_train = ordenc.fit_transform(x_train)
x_test = ordenc.transform(x_test)

gnb = GaussianNB()
gnb.fit(x_train, y_train)
gnb.score(x_test, y_test)
#otteniamo 0.34, considerato 5 classi, random è attorno 20%, quindi un pelo meglio della casualità

#proviamo invece il categorial...
catnb = CategoricalNB()
catnb.fit(x_train, y_train)
catnb.score(x_test, y_test)
#otteniamo valore migliore!

#Naive Bayes (esercitazione)

Spam filtering

In [ ]:
from sklearn.datasets import load_iris, make_gaussian_quantiles, make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, classification_report, confusion_matrix
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB

RANDOM_STATE = 0

#preprocessing step
df = pd.read_csv("spam.csv", encoding="ISO-8859-1") #usiamo encoding apposito perchè il dataset è molto sporco come caratteri
df.head() #target "v1" = ham oppure spam, "v2" variabile target, il resto non importa

df = df[["v2","v1"]]
df = df.rename(columns={"v2": "MESSAGE", "v1":"SPAM"})
df.head()

#endoding della variabile target
classes_encoding = {"spam":1, "ham": 0}
df["SPAM"] = df["SPAM"].map(lambda x: classes_encoding[x])
df.head()

#hold out
x = df["MESSAGE"].values
y = df["SPAM"].values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.3, random_state=RANDOM_STATE)
x_train.shape

#iniziamo con modello Bernnoulli (feature descritte da distribuzione binaria)
#usiamo fuzione HashingVectorizer che converte testo in matrice di token
from sklearn.feature_extraction.text import HashingVectorizer

enc = HashingVectorizer(n_features=1000, stop_words="english")
x_train = enc.fit_transform(x_train) #matrice sparsa composta per lo più da zeri
x_test = enc.transform(x_test)

x_train[0:].toarray() #per vederla come matrice classica

bnb = BernoulliNB()
bnb.fit(x_train, y_train)
print(classification_report(y_test, bnb.predict(x_test))) #il modello non è male, vediamo con multinomiale...

#proseguiamo analisi con modello di Bernoulli
#dobbiamo trattare token per frequenza e non solo on modalità binaria
from sklearn.feature_extraction.text import CountVectorizer

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.3, random_state=RANDOM_STATE)

enc = CountVectorizer(max_features=1000, stop_words="english")
x_train = enc.fit_transform(x_train)
x_test = enc.transform(x_test)

mnb = MultinomialNB()
mnb.fit(x_train, y_train)
print(classification_report(y_test, mnb.predict(x_test))) #il modello è anche migliore del precedente...

#non abbiamo controllato se il dataset era bilanciato o no sul target...
df["SPAM"].value_counts() #molto sbilanciato...

#...quindi vale la pena testare con il complement
from sklearn.naive_bayes import ComplementNB

cnb = ComplementNB()
cnb.fit(x_train, y_train)
print(classification_report(y_test, mnb.predict(x_test))) #in questo caso era meglio il multinomial, troppo penalizzata la classe predominante

#proviamo con 10000 feature invece di 1000...c'è un ulteriore miglioramento

#test pratico del modello...

def is_spam(text):
  text_enc = enc.transform([text])
  pred = mnb.predict(text_enc)[0] #primo e unico valore
  return pred==1

is_spam("Sex sex sex") #true
is_spam("nature air water") #false


#Support Vector Machines (teoria)

###Maximal margin classifier

In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs

RANDOM_SEED = 6

x,y = make_blobs(n_samples=40, centers=2, random_state=RANDOM_SEED) #dataset di prova con 2 classi
plt.scatter(x[:,0], x[:,1], c=y, s=30, cmap=plt.cm.Paired)

#usiamo un SVM lineare (non esiste funzione Maximal margin classifier in sklearn)
from sklearn.svm import SVC

model = SVC(kernel="linear")
model.fit(x, y)
print(model.support_vectors_)

#per unire i due grafici....
ax = plt.gca()

#rapprestinamo i soft margin, così vediamo i vettori di supporto
#senza (plot_method="contour") i colori mostrano le probabilità di appartenenza alle due classi in base alle sfumature
from sklearn.inspection import DecisionBoundaryDisplay
DecisionBoundaryDisplay.from_estimator(
    model,
    x,
    ax=ax,
    alpha=0.5,
    plot_method="contour",
    levels=[-1,0,1],
    linestyles=["--","-","--"],
    colors="k"
)

ax.scatter(
    model.support_vectors_[:,0],
    model.support_vectors_[:,1],
    s=100,
    linewidth=1,
    facecolors = "none",
    edgecolors = "k"
)


###Linear SVM

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

RANDOM_SEED = 0

In [ ]:
def plot_decision_boundary(model, data, sv=None):

    X, Y = data
    h = .02

    x_min, x_max = X[:, 0].min()-.1, X[:, 0].max()+.1
    y_min, y_max = X[:, 1].min()-.1, X[:, 1].max()+.1

    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))

    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])

    Z = Z.reshape(xx.shape)
    plt.contourf(xx, yy, Z, cmap=plt.cm.Paired)

    X_m = X[Y==1]
    X_b = X[Y==0]
    plt.scatter(X_b[:, 0], X_b[:, 1], c="green",  edgecolor='white')
    plt.scatter(X_m[:, 0], X_m[:, 1], c="red",  edgecolor='white')

    if sv is not None:
          plt.scatter(sv[:, 0], sv[:, 1], c="blue",  edgecolor='white')

def classifier_report(model, data):
  X, y = data
  y_pred = model.predict(X)
  report = classification_report(y_pred, y)
  print(report)

In [ ]:
#generiamo un dataset...
X, y = make_classification(n_samples=100, n_features=2, n_informative=2, n_redundant=0, n_repeated=0, n_classes=2, random_state=RANDOM_SEED)
plt.scatter(X[:,0], X[:,1], c=y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=RANDOM_SEED)

In [ ]:
#creiamo SVM lineare (che non ha proprietà del kernel)
from sklearn.svm import LinearSVC

svc = LinearSVC()
svc.fit(X_train, y_train)
classifier_report(svc, (X_test, y_test)) #il modello è molto buono

plot_decision_boundary(svc, (X_train, y_train))

#vediamo ora i support vector usando SVC
from sklearn.svm import SVC

svc = SVC(kernel="linear")
svc.fit(X_train, y_train)
classifier_report(svc, (X_test, y_test))

plot_decision_boundary(svc, (X_train, y_train), sv=svc.support_vectors_)
#molti elementi, cioè tutti quella nella sezione sbagliata...

#vediamo cosa succede toglierndo outlier "a occhio"...
X_train[:,0]>3.5
outlier_index = np.where(X_train[:,0]>3)
X_train[outlier_index,:]
X_train2 = np.delete(X_train, outlier_index, axis=0)
y_train2 = np.delete(y_train, outlier_index)

X_train2.shape #68 elementi
X_train.shape #70 elementi

#come  metriche è uguale..
svc = SVC(kernel="linear")
svc.fit(X_train2, y_train2)
classifier_report(svc, (X_test, y_test))

plot_decision_boundary(svc, (X_train2, y_train2), sv=svc.support_vectors_)

#ora proviamo caso estremo, rimuovendo tutti gli outliers utilizzando lo z-score > deviazione standard
from scipy import stats

outliers_map = np.abs(stats.zscore(X_train)) < X_train.std(axis=0)
outliers_map = outliers_map[:,0] & outliers_map[:,1]

X_train_filtered = X_train[outliers_map]
y_train_filtered = y_train[outliers_map]

X_train_filtered.shape #48 elementi rimasti

svc = SVC(kernel="linear")
svc.fit(X_train_filtered, y_train_filtered)
classifier_report(svc, (X_test, y_test))

plot_decision_boundary(svc, (X_train_filtered, y_train_filtered), sv=svc.support_vectors_)

###Esempio SVM con vari Kernel

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

RANDOM_SEED = 0

In [ ]:
def plot_decision_boundary(model, train_set, test_set, sv=None):

    #plt.figure(figsize=figsize)

    if(model):
        X_train, Y_train = train_set
        X_test, Y_test = test_set
        X = np.vstack([X_train, X_test])
        x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
        y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5

        xx, yy = np.meshgrid(np.arange(x_min, x_max, .02),
                             np.arange(y_min, y_max, .02))

        if hasattr(model, "predict_proba"):
            Z = model.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]
        else:
            Z = model.predict(np.c_[xx.ravel(), yy.ravel()])

        Z = Z.reshape(xx.shape)

        plt.contourf(xx, yy, Z, alpha=.8)

    plt.scatter(X_train[:,0], X_train[:,1], c=Y_train)
    plt.scatter(X_test[:,0], X_test[:,1], c=Y_test, alpha=0.6)

    if sv is not None:
      plt.scatter(sv[:, 0], sv[:, 1], c="blue",  edgecolor='white')

    plt.show()

def classifier_report(model, data):
  X, y = data
  y_pred = model.predict(X)
  report = classification_report(y_pred, y)
  print(report)

In [ ]:
from sklearn.datasets import make_circles

X, y = make_circles(noise=0.2, factor=0.5, random_state=RANDOM_SEED) #generazione di un dataset con parte circolare all'interno
plt.scatter(X[:,0],X[:,1],c=y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=RANDOM_SEED)

In [ ]:
#kerner lineare
svc = SVC(kernel="linear", probability=True) #possibile richiedere anche la probabilità
svc.fit(X_train, y_train)
classifier_report(svc, (X_test, y_test)) #pessimo modello...

plot_decision_boundary(svc, (X_train, y_train), (X_test, y_test), sv=svc.support_vectors_)

In [ ]:
#kerner polinomiale
svc = SVC(kernel="poly", probability=True)
svc.fit(X_train, y_train)
classifier_report(svc, (X_test, y_test))

plot_decision_boundary(svc, (X_train, y_train), (X_test, y_test), sv=svc.support_vectors_) #un pò meglio del precedente

In [ ]:
#kernel sigmoidale
from sklearn.svm import SVC

svc = SVC(kernel="sigmoid", probability=True)
svc.fit(X_train, y_train)
classifier_report(svc, (X_test, y_test))

plot_decision_boundary(svc, (X_train, y_train), (X_test, y_test))
#si comporta male, adatto a casi in cui la funzione è sigmoidale, quindi oscillante

In [ ]:
#kernel gaussiano
from sklearn.svm import SVC

svc = SVC(kernel="rbf", probability=True)
svc.fit(X_train, y_train)
classifier_report(svc, (X_test, y_test))

plot_decision_boundary(svc, (X_train, y_train), (X_test, y_test)) #la migliore, con i valori che tendono a raggrupparsi

#Support Vector Machines (esercitazione)

Previsione infarto cardiaco

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

RANDOM_STATE = 0
BASE_URL = "https://raw.githubusercontent.com/ProfAI/machine-learning-modelli-e-algoritmi/main/datasets/heart.csv"

df = pd.read_csv(BASE_URL)
df.head()

#valutazione dei dati:
#one hot encoding per varibili categoriche non ordinate
 #scale diverse sarà  da standardizzare
df.describe()

X = df.drop(["output"], axis=1).values
y = df["output"].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=RANDOM_STATE)
X_train.shape()
X_test.shape()

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(
    [
        ("ohe", OneHotEncoder(), [2]),
    ],
    remainder="passthrough" # per non eliminare le altre colonne
)

X_train = ct.fit_transform(X_train) #quindi abbiamo ora le colonne in più
X_test = ct.transform(X_test)

std = StandardScaler()
X_train = std.fit_transform(X_train)
X_test = std.transform(X_test)
print (X_train.mean(), X_train.std()) #media intorno allo zero e std a 1

In [ ]:
def classifier_report(model, data):
  X, y = data
  y_pred = model.predict(X)
  report = classification_report(y, y_pred)
  print(report)
  print(confusion_matrix(y, y_pred))

In [ ]:
#Modello baseline - Regressione logistica
lr = LogisticRegression()
lr.fit(X_train, y_train)
classifier_report(lr, (X_test, y_test))
#non è male, ma dobbiamo attenzionare mancati avvisi di infarti
#quindi precision ci dice quanti positivi erano positivi, invece recall ci dice i casi veramente riconosciuti quindi FalseNegative, in questo caso vale 4...

#Support Vector Classifier lineare (probabimente avrà comportamento simile al precedente)
svc = SVC(kernel="linear")
svc.fit(X_train, y_train)
classifier_report(svc, (X_test, y_test))
#uguale al preceente...proseguiamo con altri tentativi...

#kernel SVM
svc = SVC(kernel="rbf") #gaussiano
svc.fit(X_train, y_train)
classifier_report(svc, (X_test, y_test))
#ancora due casi....proviamo a applicare pesi per penalizzare...
svc = SVC(kernel="rbf", class_weight={1:1.5, 0:0.5})
svc.fit(X_train, y_train)
classifier_report(svc, (X_test, y_test))
#non basta e precisione peggiorata...probabilmente ci sono outliers tra le due classi

#si potrebbe usare un valore di soglia più basso, invece di 0.5 che è il default

#Neural Networks (teoria)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

!wget https://raw.githubusercontent.com/ProfAI/machine-learning-modelli-e-algoritmi/main/script/viz.py

from viz import plot_decision_boundary
RANDOM_SEED = 0

#generazione dei dati con relazione molto complessa su due classi
#non c'è nessuna retta di separazione tra le due classi
X, y = make_moons(n_samples=100, shuffle=True, noise=0.25, random_state=RANDOM_SEED)
plt.scatter(X[:,0], X[:,1], c=y)

#preprocessing dei dati
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=RANDOM_SEED)

#normalizzazione
mms = MinMaxScaler()
x_train = mms.fit_transform(x_train)
x_test = mms.transform(x_test)

#esperimento con modello lineare, il perceptor
from sklearn.linear_model import Perceptron
model = Perceptron() #unica differenza da regressione logistica è che quella usa sigmoide come funzione attivazione finale, qui si usa step function
model.fit(x_train, y_train)
model.score(x_test, y_test) #accuracy, 0.85, retta che separa parzialmente le due classi...
plot_decision_boundary(model, (x_train, y_train), (x_test, y_test)) #retta senza soft margin, il percettrone è 0 o 1 data la step function

#esperimento con regressione logistica lineare, che però tiene conto probabilità output
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr.score(X_test, y_test) #sempre 0.85
plot_decision_boundary(lr, (x_train, y_train), (x_test, y_test))

#proviamo multi-layer perceptor
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(100,100,100,), random_state=RANDOM_SEED) #3 strati con 100 nodi ciascuno
mlp.fit(X_train, y_train)
mlp.score(X_test, y_test) #0.9
plot_decision_boundary(mlp, (x_train, y_train), (x_test, y_test)) #il modello commette piccoli errori, ma approssima meglio la distribuzione

#sezioniamo la rete per vedere i pesi...
len(mlp.coefs_) #numero strati nascosti + input
mlp.coefs_[0] #pesi tra strato input e primo nascosto
len(mlp.intercepts_) #bias di ogni strato

weights = 0
for i in range(len(mlp.coefs_)):
  coefs = mlp.coefs_[i].size
  bias = mlp.intercepts_[i].size

  tot = coefs+bias
  print(f"Layer {i} => Pesi={coefs} Bias={bias} Totale={tot}")

  weights += tot

print(f"Pesi totali della rete={weights}")
#al primo step 200 pesi, 2 feature di ingresso, ognuna a 100 neuroni
#al secondo step 100 neuroni che si collegano ad altri 100, quindi 10000, bias rimane sempre uno per quello successivo
#l'ultimo strato nascosto con l'output avrà 100 pesi perchè è uno l'output
#le reti neurali non sono interpretabili e richiedono molti dati!

#a cosa serve la funzione di attivazione?
#proviamo a toglierla, tornando x=x
hiddel_layer_size = (100,)*10 #10 strati nascosti con 100 nodi
mlp = MLPClassifier(hidden_layer_sizes=hiddel_layer_size, activation="identity", random_state=RANDOM_SEED)
mlp.fit(X_train, y_train)
mlp.score(X_test, y_test) #0.85
plot_decision_boundary(mlp, (x_train, y_train), (x_test, y_test))
#togliendo funzione di attivazione, abbiamo tolto possibilità di avere funzioni non lineari, quindi modello lineare ricade in quello visto sopra

#Neural Networks (esercitazione)

Classificazione di cifre scritte a mano

In [ ]:
#l'ultimo elemento è la classe di appartenenza, i precedenti sono il valore numerico dell'immagine (8x8 pixel)
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/optdigits/optdigits.tra
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/optdigits/optdigits.tes

In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

RANDOM_SEED = 0

### Caricamento dei dati...
from numpy import genfromtxt

arr = genfromtxt('optdigits.tra', delimiter=',')
arr.shape #3823 righe e 65 colonne (da 0 a 63 i valori dei pixel, l'ultimo è il target)

#stampiamo un immagine di esempio
sample_num = 100
print(arr[sample_num,-1])
plt.imshow(arr[sample_num,:-1].reshape(8,8))

### Processiamo i dati...
x = arr[:,:-1] #3823,64
y = arr[:,-1] #3823,

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.2, random_state=RANDOM_SEED)
x_train.max() #max è 16, quindi non dividiamo per 255 ma 16 in questo caso
x_max = x_train.max()
x_train/=x_max
x_test/=x_max
x_train.max() #max è normalizzato a 1

### Multilayer Perceptron...
mlp = MLPClassifier(random_state=RANDOM_SEED)
mlp.fit(x_train, y_train)
mlp.score(x_test, y_test) #0.97, accuracy molto buona
y_pred = mlp.predict(x_test)
print(classification_report(y_test, y_pred))
#avendo più classi abbiamo spaccato per ognuna

#ricaviamo le immagini che non sono state interpretate correttamente...
errors_mask = y_pred!=y_test
x_errors = x_test[errors_mask]
y_errors = y_test[errors_mask]
y_errors_pred = y_pred[errors_mask]
x_errors.shape #18 errori

fig = plt.figure(figsize=(12,12))
fig.subplots_adjust(wspace=0, hspace=0)

for i in range(x_errors.shape[0]):
  plot = fig.add_subplot(4, 5, i+1)
  plt.imshow(x_errors[i,:].reshape(8,8))
  plot.text(0, 7, f"Class: {int(y_errors[i])}", fontsize=14, fontdict={'weight': 'bold'})
  plot.text(0, 6, f"Predicted: {int(y_errors_pred[i])}", fontsize=14, fontdict={'weight': 'bold'})
  plt.axis("off")

###Proviamo cosa sarebbe successo con regressione logistica semplice
lr = LogisticRegression()
lr.fit(x_train, y_train)
lr.score(x_test, y_test)
#accuracy del 0.97

#K-Nearest Neighbors (teoria)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_classification

RANDOM_SEED = 0

In [ ]:
#classificazione binaria di test
X, y = make_classification(n_samples=100, n_features=2, n_informative=2, n_redundant=0, n_repeated=0, n_classes=2, random_state=RANDOM_SEED)
plt.scatter(X[:,0], X[:,1], c=y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=RANDOM_SEED)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

Ks = [1,2,3,4,5,7,10,12,15,20,30,40,50] # proviamo vari valori di K

for K in Ks:

    print("K="+str(K))
    knn = KNeighborsClassifier(n_neighbors=K)
    knn.fit(X_train, y_train)

    y_pred_train = knn.predict(X_train)
    y_prob_train = knn.predict_proba(X_train)

    y_pred = knn.predict(X_test)
    y_prob = knn.predict_proba(X_test)

    accuracy_train = accuracy_score(y_train, y_pred_train)
    accuracy_test = accuracy_score(y_test, y_pred)

    loss_train = log_loss(y_train, y_prob_train)
    loss_test = log_loss(y_test, y_prob)

    print("ACCURACY: TRAIN=%.4f TEST=%.4f" % (accuracy_train,accuracy_test))
    print("LOG LOSS: TRAIN=%.4f TEST=%.4f" % (loss_train,loss_test))

#10 pare il migliore valore, testiamolo....
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train, y_train)
x = [0,0] #punto centrale grafico
y_pred = knn.predict([x])
y_pred #predetto classe gialla, giusto

#viualizziamo i valori vicini...
distances, neighbors = knn.kneighbors([x])
neighbors #indici nel dataset dei valori più vicini

X_neighbors = X_train[neighbors][0]

plt.scatter(X[:,0], X[:,1], c=y)
plt.scatter(x[0], y[0], c="green")
plt.scatter(X_neighbors[:,0], X_neighbors[:,1], facecolors='none', edgecolors='r')

#solo uno viola, infatti:
knn.predict_proba([x]) #array([[0.1, 0.9]])

###Radius Nearest Neighbors

Siccome basato su distanze fonadmentale normalizzare

In [ ]:
mms = MinMaxScaler()
X_train_norm = mms.fit_transform(X_train)
X_test_norm = mms.transform(X_test)

from sklearn.neighbors import RadiusNeighborsClassifier

Rs = [.2, .3, .4, .5, .6, .7, .8, .9, 1]

for R in Rs:

    print("Radius="+str(R))
    rnn = RadiusNeighborsClassifier(radius=R)
    rnn.fit(X_train_norm, y_train)

    y_pred_train = rnn.predict(X_train_norm)
    y_prob_train = rnn.predict_proba(X_train_norm)

    y_pred = rnn.predict(X_test_norm)
    y_prob = rnn.predict_proba(X_test_norm)

    accuracy_train = accuracy_score(y_train, y_pred_train)
    accuracy_test = accuracy_score(y_test, y_pred)

    loss_train = log_loss(y_train, y_prob_train)
    loss_test = log_loss(y_test, y_prob)

    print("ACCURACY: TRAIN=%.4f TEST=%.4f" % (accuracy_train,accuracy_test))
    print("LOG LOSS: TRAIN=%.4f TEST=%.4f" % (loss_train,loss_test))

#il migliore è con raggio .2
r = .2
rnn = RadiusNeighborsClassifier(radius=r)
rnn.fit(X_train_norm, y_train)

#mostriamo il raggio..
x = [0,0]
x = mms.transform([x])
y_pred = rnn.predict(x)
y_pred

fig, ax = plt.subplots() # note we must use plt.subplots, not plt.subplot

ax.scatter(X_train_norm[:,0], X_train_norm[:,1], c=y_train)
ax.scatter(x[0,0], x[0,1], c="green")

circle = plt.Circle((x[0,0], x[0,1]), r, color='r', fill=None)

ax.add_patch(circle)
plt.show()

##K-Nearest Neighbors (esercitazione)

Face recognition (richiesta accuracy del 80%)

In [ ]:
#scarichiamo il dataset con immagini (rappresentati come numeri)
#c'è una cartella per persona, con 10 immagini della persona da varie angolazioni
!wget https://github.com/ProfAI/machine-learning-modelli-e-algoritmi/raw/main/datasets/olivetti_faces.zip
!unzip olivetti_faces.zip

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from os import listdir
import cv2

RANDOM_SEED = 0
img_size = (64, 64)

In [ ]:
img = cv2.imread("olivetti_faces/0/0.jpg", cv2.IMREAD_GRAYSCALE)
img.shape # (64, 64, 3) base, altezza canale gray
plt.imshow(img, cmap="gray")

In [ ]:
FOLDER = "olivetti_faces"

X = []
y = []

for dir in listdir(FOLDER):
  path = FOLDER+"/"+dir+"/"
  for f in listdir(path):
    if ".jpg" in f:
      x = cv2.imread(path+f, cv2.IMREAD_GRAYSCALE)
      X.append(x)
      y.append(dir)

X = np.array(X)
y = np.array(y)

X.shape

In [ ]:
plt.imshow(X[0], cmap="gray")

In [ ]:
X = X.reshape(X.shape[0], X.shape[1]*X.shape[2]) #in modo da avere un unica dimenisone, così abbiamo 400 osservazione con 4096 pixel (64*64)
X.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=RANDOM_SEED)
X_train = X_train/255 #per le immagini basta normalizzare facendo divisione per 255 in modo da avere valori tra 0 e 1
X_test = X_test/255

#potrebbe succedere che una classe non venga messa nel train ma solo nel test, anche se di solito train_test_split si occupa di questo
(1-np.isin(y_test, y_train)).sum()

In [ ]:
#rilanciamo valutazione della teoria e scegliamo k=3 come migliore tra accuracy (evitando overfitting)
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

knn.score(X_test, y_test)

#per testare con una foto, necessario ritagliare solo il volto
!wget https://www.antoniocapraro.it/public/Files/rif000002/135/viso-perfetto-4.jpg #da ridimensionare
x = cv2.imread("viso-perfetto-4.jpg", cv2.IMREAD_GRAYSCALE)
plt.imshow(x, cmap="gray")

x = x.reshape(img_size[0]*img_size[1])
x = x/255

x.shape

y_pred = knn.predict([x])
y_pred

y_proba = knn.predict_proba([x])[0]

#non emerge nessun sosia
for i in range(y_proba.shape[0]):
  print(f"Person {knn.classes_[i]} = {y_proba[i]} probability")